In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Not connected to a GPU


In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


# Atividade final avaliativa da disciplina de Processamento de Dados e Longa Escala 
## **Tutor**: Anderson Felipe Rocha
##Alunos**: Caio Serpa, Eden Coelho e Roberto Sá


#Configuração do ambiente, instalação do hadoop e spark

In [3]:
#instalar o Java 8 na maquina da sessão
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

#setando jdk 8
#!sudo apt install openjdk-8-jdk
#!sudo update-alternatives --config java 

#spark
!wget -q https://ftp.unicamp.br/pub/apache/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
#extração do spark
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
#findspark para facilitar na criação da sessão spark
!pip install -q findspark=
!pip install  pyspark==3.1.2

ERROR: Invalid requirement: 'findspark='
Hint: = is not a valid operator. Did you mean == ?
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!java -version

openjdk version "11.0.17" 2022-10-18
OpenJDK Runtime Environment (build 11.0.17+8-post-Ubuntu-1ubuntu218.04)
OpenJDK 64-Bit Server VM (build 11.0.17+8-post-Ubuntu-1ubuntu218.04, mixed mode, sharing)


In [5]:
import os
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

#variáveis de ambiente
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"



os.environ['PYSPARK_SUBMIT_ARGS'] = '\
    --driver-memory 20G \
    --executor-memory 20G \
    pyspark-shell'



print(os.environ['JAVA_HOME'])
print(os.environ['SPARK_HOME']) 

/usr/lib/jvm/java-8-openjdk-amd64
/content/spark-3.1.2-bin-hadoop3.2


In [6]:
!pip install findspark
import findspark

findspark.find()
findspark.init()

conf = SparkConf().setMaster("local[*]")
sc = SparkContext.getOrCreate(conf=conf)
#sc.stop()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Importação do conjunto de dados

Fonte dos dados: https://www.kaggle.com/datasets/najzeko/steam-reviews-2021?resource=download

Tamanho do arquivo: 8.17GB
Formato do arquivo: CSV

Conjunto de dados de cerca de 21 milhões de avaliações de usuários de cerca de 300 jogos diferentes no Steam. Obtido usando a API fornecida pelo Steam descrita na documentação do Steamworks

In [8]:
#recebendo os dados via api do google drive
from google.colab import drive 

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [9]:
from pyspark.sql.functions import to_timestamp
spark = SparkSession \
    .builder \
    .getOrCreate()


#data_spark = spark.read.csv('gdrive/My Drive/Colab Notebooks/steam_reviews.csv', header=True, inferSchema=False)

In [10]:
from pyspark.sql.types import *

schema = StructType([
    StructField('_c0', IntegerType(), False),
    StructField('app_id',IntegerType(), False),
    StructField('app_name', StringType(),False),
    StructField('review_id',IntegerType(), False),
    StructField('language', StringType(),False),
    StructField('review', StringType(),False),
    StructField('timestamp_created', LongType(), False),
    StructField('timestamp_updated', LongType(), False),
    StructField('recommended', BooleanType(), False),
    StructField('votes_helpful',IntegerType(), False),
    StructField('votes_funny',IntegerType(), False),
    StructField('weighted_vote_score', FloatType(), False),
    StructField('comment_count',IntegerType(), False),
    StructField('steam_purchase', BooleanType(), False),
    StructField('received_for_free', BooleanType(), False),
    StructField('written_during_early_access', BooleanType(), False),
    StructField('author.steamid', LongType(), False),
    StructField('author.num_games_owned',IntegerType(), False),
    StructField('author.num_reviews',IntegerType(), False),
    StructField('author.playtime_forever', FloatType(), False),
    StructField('author.playtime_last_two_weeks', FloatType(), False),
    StructField('author.playtime_at_review', FloatType(), False),
    StructField('author.last_played', StringType(), False),
])


In [11]:
data_spark = spark.read.csv('/content/gdrive/MyDrive/Colab Notebooks/steam_reviews.csv', header=True, schema=schema)

#data_spark = spark.read.csv('gdrive/My Drive/Colab Notebooks/steam_reviews.csv', header=True, inferSchema=True)

In [12]:
data_spark.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- app_id: integer (nullable = true)
 |-- app_name: string (nullable = true)
 |-- review_id: integer (nullable = true)
 |-- language: string (nullable = true)
 |-- review: string (nullable = true)
 |-- timestamp_created: long (nullable = true)
 |-- timestamp_updated: long (nullable = true)
 |-- recommended: boolean (nullable = true)
 |-- votes_helpful: integer (nullable = true)
 |-- votes_funny: integer (nullable = true)
 |-- weighted_vote_score: float (nullable = true)
 |-- comment_count: integer (nullable = true)
 |-- steam_purchase: boolean (nullable = true)
 |-- received_for_free: boolean (nullable = true)
 |-- written_during_early_access: boolean (nullable = true)
 |-- author.steamid: long (nullable = true)
 |-- author.num_games_owned: integer (nullable = true)
 |-- author.num_reviews: integer (nullable = true)
 |-- author.playtime_forever: float (nullable = true)
 |-- author.playtime_last_two_weeks: float (nullable = true)
 |-- author.pl

In [13]:
data_spark = data_spark.withColumnRenamed('_c0', 'id')\
            .withColumnRenamed('author.steamid', 'author_steamid')\
            .withColumnRenamed('author.num_games_owned', 'author_num_games_owned')\
            .withColumnRenamed('author.num_reviews', 'author_num_reviews')\
            .withColumnRenamed('author.playtime_forever', 'author_playtime_forever')\
            .withColumnRenamed('author.playtime_last_two_weeks', 'author_playtime_last_two_weeks')\
            .withColumnRenamed('author.playtime_at_review', 'author_playtime_at_review')\
            .withColumnRenamed('author.last_played', 'author_last_played')



In [14]:
#data_spark.show(5)

In [15]:
data_spark = data_spark.na.drop()
data_spark.count()

16619206

In [16]:
data_spark.show(10,truncate=False)

+---+------+------------------------+---------+--------+-----------------------------------------------------------+-----------------+-----------------+-----------+-------------+-----------+-------------------+-------------+--------------+-----------------+---------------------------+-----------------+----------------------+------------------+-----------------------+------------------------------+-------------------------+------------------+
|id |app_id|app_name                |review_id|language|review                                                     |timestamp_created|timestamp_updated|recommended|votes_helpful|votes_funny|weighted_vote_score|comment_count|steam_purchase|received_for_free|written_during_early_access|author_steamid   |author_num_games_owned|author_num_reviews|author_playtime_forever|author_playtime_last_two_weeks|author_playtime_at_review|author_last_played|
+---+------+------------------------+---------+--------+----------------------------------------------------

In [17]:
#data_spark.count()

In [18]:
from pyspark.sql.functions import *

#função para remover o ponto e o zero da coluna autor_last_played
get_number_without_dot = udf(lambda s: s.split('.')[0], StringType())
data_spark = data_spark.withColumn('s_author_last_played', get_number_without_dot(data_spark.author_last_played))
data_spark.count()

16619206

In [19]:
data_spark.show(10,truncate=False)

+---+------+------------------------+---------+--------+-----------------------------------------------------------+-----------------+-----------------+-----------+-------------+-----------+-------------------+-------------+--------------+-----------------+---------------------------+-----------------+----------------------+------------------+-----------------------+------------------------------+-------------------------+------------------+--------------------+
|id |app_id|app_name                |review_id|language|review                                                     |timestamp_created|timestamp_updated|recommended|votes_helpful|votes_funny|weighted_vote_score|comment_count|steam_purchase|received_for_free|written_during_early_access|author_steamid   |author_num_games_owned|author_num_reviews|author_playtime_forever|author_playtime_last_two_weeks|author_playtime_at_review|author_last_played|s_author_last_played|
+---+------+------------------------+---------+--------+----------

In [20]:
#Convertendo para longType() a coluna s_author_last played
data_spark = data_spark.withColumn('s_author_last_played',data_spark["s_author_last_played"].cast(LongType()))
data_spark.count()
#Convertendo para data as seguintes colunas
data_spark  = data_spark.withColumn('s_author_last_played', from_unixtime(col('s_author_last_played')))\
          .withColumnRenamed('s_author_last_played','t_author_last_played')\
          .withColumn('t_timestamp_created', from_unixtime(col('timestamp_created')))\
          .withColumn('t_timestamp_updated', from_unixtime(col('timestamp_updated')))\
       

#dropando colunas
cols = ("timestamp_created","timestamp_updated","author_last_played")
data_spark = data_spark.drop(*cols)
data_spark.count()

data_spark.show(10,truncate=False)

+---+------+------------------------+---------+--------+-----------------------------------------------------------+-----------+-------------+-----------+-------------------+-------------+--------------+-----------------+---------------------------+-----------------+----------------------+------------------+-----------------------+------------------------------+-------------------------+--------------------+-------------------+-------------------+
|id |app_id|app_name                |review_id|language|review                                                     |recommended|votes_helpful|votes_funny|weighted_vote_score|comment_count|steam_purchase|received_for_free|written_during_early_access|author_steamid   |author_num_games_owned|author_num_reviews|author_playtime_forever|author_playtime_last_two_weeks|author_playtime_at_review|t_author_last_played|t_timestamp_created|t_timestamp_updated|
+---+------+------------------------+---------+--------+----------------------------------------

In [21]:
#data_spark.coalesce(1).write.option("header",True).format("csv").save("gdrive/MyDrive/clean_data")

In [26]:
data_spark.cache()
data_spark.show()

+---+------+--------------------+---------+--------+-------------------------------------+-----------+-------------+-----------+-------------------+-------------+--------------+-----------------+---------------------------+-----------------+----------------------+------------------+-----------------------+------------------------------+-------------------------+--------------------+-------------------+-------------------+
| id|app_id|            app_name|review_id|language|                               review|recommended|votes_helpful|votes_funny|weighted_vote_score|comment_count|steam_purchase|received_for_free|written_during_early_access|   author_steamid|author_num_games_owned|author_num_reviews|author_playtime_forever|author_playtime_last_two_weeks|author_playtime_at_review|t_author_last_played|t_timestamp_created|t_timestamp_updated|
+---+------+--------------------+---------+--------+-------------------------------------+-----------+-------------+-----------+-------------------+

## CRIANDO TEMP VIEW

In [27]:
data_spark.createOrReplaceTempView("temp_steam")
data_spark.show(5)

+---+------+--------------------+---------+--------+----------------------------------+-----------+-------------+-----------+-------------------+-------------+--------------+-----------------+---------------------------+-----------------+----------------------+------------------+-----------------------+------------------------------+-------------------------+--------------------+-------------------+-------------------+
| id|app_id|            app_name|review_id|language|                            review|recommended|votes_helpful|votes_funny|weighted_vote_score|comment_count|steam_purchase|received_for_free|written_during_early_access|   author_steamid|author_num_games_owned|author_num_reviews|author_playtime_forever|author_playtime_last_two_weeks|author_playtime_at_review|t_author_last_played|t_timestamp_created|t_timestamp_updated|
+---+------+--------------------+---------+--------+----------------------------------+-----------+-------------+-----------+-------------------+---------

##PERGUNTA 1 - Qual usuário é mais deu reviews em jogos?

In [28]:
query = """SELECT author_steamid, count(author_steamid) as Reviews
 FROM temp_steam 
 GROUP BY author_steamid
 ORDER BY Reviews DESC
 limit 1
 """

spark.sql(query).show()

+-----------------+-------+
|   author_steamid|Reviews|
+-----------------+-------+
|76561198315585536|    128|
+-----------------+-------+



## PERGUNTA 2 - Qual jogo é mais recomendado e qual jogo é o menos recomendado?

Jogo mais rcomendado

In [29]:
query = """ SELECT app_name, COUNT(app_id) as qtd_recommended FROM temp_steam
WHERE recommended == true
GROUP BY app_name
ORDER BY qtd_recommended DESC
limit 1
"""
spark.sql(query).show()

+--------------------+---------------+
|            app_name|qtd_recommended|
+--------------------+---------------+
|PLAYERUNKNOWN'S B...|         736012|
+--------------------+---------------+



Jogo menos recomendado

In [30]:
query = """ SELECT app_name, COUNT(app_id) as qtd_recommended FROM temp_steam
WHERE recommended == true
GROUP BY app_name
ORDER BY qtd_recommended ASC
limit 1
"""
spark.sql(query).show()

+--------+---------------+
|app_name|qtd_recommended|
+--------+---------------+
|     000|              2|
+--------+---------------+



##PERGUNTA 3 - Qual a variedade de linguas em reviews?

In [31]:
query = """ SELECT DISTINCT(language) FROM temp_steam""" 
number = spark.sql(query).count()
spark.sql(query).show(number)

+----------+
|  language|
+----------+
|   koreana|
|     greek|
|   russian|
|    danish|
|     dutch|
|  tchinese|
|    german|
|   spanish|
|    french|
|vietnamese|
|  schinese|
|   italian|
|   swedish|
|      thai|
| bulgarian|
|   turkish|
|   finnish|
|portuguese|
|  japanese|
| ukrainian|
|   english|
|    polish|
|     latam|
|       000|
| hungarian|
|     czech|
|  romanian|
| norwegian|
| brazilian|
+----------+



In [32]:
print('Existem": ' + str(number) + ' idiomas diferentes nas avaliações')

Existem": 29 idiomas diferentes nas avaliações
